In [1]:
import pandas as pd
import numpy as np
from watertap3.utils import watertap_setup, get_case_study, run_model 
from watertap3.utils import run_watertap3, run_model_no_print, run_and_return_model
import pyomo.util.infeasible as infeas
from idaes.core.util.model_statistics import *
import matplotlib.pyplot as plt
from truck_pipe_cost_functions import truck_costing, pipe_costing
from watertap3.utils.post_processing import get_results_table

In [2]:
# Assuming that the LCOW calculated for each unit process has the treated volume as the basis
# 99.999% sure

In [3]:
# flow_list = np.array([0.0075,0.01,0.015,0.02,0.05,0.1,0.2])#,0.225,0.45]) #m3/s
flow_list = np.array([0.0075,0.22875, 0.45])
conc_list = np.array([0.5,9.75,19])*10
dist_list = np.array([10,50,100,200])

In [4]:
for flow in flow_list:
    for dist in dist_list:
        print(flow,dist,pipe_costing(flow*3600*24,dist))

N: 2 lx: 4242.087383316288
0.0075 10 0.4831302997963953
N: 11 lx: 4242.087491841825
0.0075 50 1.7741624590315976
N: 23 lx: 4242.087505407518
0.0075 100 3.425141730150199
N: 47 lx: 4242.087512190363
0.0075 200 6.727100272387402
N: 0 lx: 23427.68991924614
0.22875 10 0.08428214601931382
N: 2 lx: 23427.693229274435
0.22875 50 0.2623048258303975
N: 4 lx: 23427.693643028037
0.22875 100 0.48276601599755814
N: 8 lx: 23427.693849904845
0.22875 200 0.9236883963318795
N: 0 lx: 32859.06048886161
0.45 10 0.0742438403899838
N: 1 lx: 32859.06700039212
0.45 50 0.21626540190657664
N: 3 lx: 32859.06781433362
0.45 100 0.3963207787651716
N: 6 lx: 32859.06822130438
0.45 200 0.7530602991985561


In [5]:
fuel_price_file_path = '/Users/mhardika/Documents/watertap3/WaterTAP3/watertap3/watertap3/data/fuel_costs.csv'
fuel_df = pd.read_csv(fuel_price_file_path)
fuel_price = fuel_df.loc[fuel_df['state_code']=='TX']['regular_cost'].values[0]
fuel_price

3.059

In [6]:
for flow in flow_list:
    for dist in dist_list:
        print(flow,dist,truck_costing(dist))

0.0075 10 2.421439737951622
0.0075 50 3.708014460659328
0.0075 100 5.31623286404396
0.0075 200 8.532669670813224
0.22875 10 2.421439737951622
0.22875 50 3.708014460659328
0.22875 100 5.31623286404396
0.22875 200 8.532669670813224
0.45 10 2.421439737951622
0.45 50 3.708014460659328
0.45 100 5.31623286404396
0.45 200 8.532669670813224


In [5]:
# Test Crystallizer

# case_study = 'big_spring'
# scenario = 'test'
# desired_recovery = 1
# ro_bounds = 'other' # or 'seawater'

# m = watertap_setup(case_study=case_study, scenario=scenario)
# m = get_case_study(m=m)
# m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

# dwi_lcow = np.array([])
# dwi_flow = np.array([])

# for flow in flow_list:
#     print(flow)
#     m.fs.big_spring_feed.flow_vol_in.fix(0.0075)
#     m.fs.big_spring_feed.conc_mass_in[0, 'tds'].fix(10)
#     m = run_and_return_model(m, objective=True,print_it=True)
#     print(m.fs.costing.LCOW.value(),m.fs.crystallizer.flow_vol_in[0].value)
#     dwi_lcow= np.concatenate((dwi_lcow,[m.fs.costing.LCOW.value()]))
#     dwi_flow= np.concatenate((dwi_flow,[m.fs.crystallizer.flow_vol_in[0].value]))

# dwi_lcow_brine_basis = dwi_lcow*flow_list/dwi_flow

In [6]:
# Deep Well Injection

case_study = 'big_spring'
scenario = 'dwi_check'
desired_recovery = 1
ro_bounds = 'other' # or 'seawater'

m = watertap_setup(case_study=case_study, scenario=scenario)
m = get_case_study(m=m)
m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

dwi_lcow_brine_basis = 0

for flow in flow_list:
    dwi_lcow = np.array([])
    dwi_flow = np.array([])
    for conc in conc_list:
        print(flow)
        m.fs.big_spring_feed.flow_vol_in.fix(flow)
        m.fs.big_spring_feed.conc_mass_in[0, 'tds'].fix(conc)
        m = run_and_return_model(m, objective=True,print_it=True)
        print(m.fs.costing.LCOW.value(),m.fs.deep_well_injection.flow_vol_in[0].value)
        dwi_lcow= np.concatenate((dwi_lcow,[m.fs.costing.LCOW.value()]))
        dwi_flow= np.concatenate((dwi_flow,[m.fs.deep_well_injection.flow_vol_in[0].value]))

    temp = dwi_lcow*flow/dwi_flow
    try:
        dwi_lcow_brine_basis = np.vstack((dwi_lcow_brine_basis,temp))
    except ValueError:
        dwi_lcow_brine_basis = temp
# m, df = get_results_table(m=m, case_study='high_flow', scenario=scenario)      

# dwi_a_conc_flow = m.fs.deep_well_injection.flow_vol_in[0].value
# dwi_a_total_water_recovery = m.fs.costing.system_recovery()
# sd_update_basis = (1-dwi_a_total_water_recovery)/dwi_a_total_water_recovery


Case Study = BIG SPRING
Scenario = DWI CHECK


=========================ADDING UNIT PROCESSES=========================
SOURCE
DEEP WELL INJECTION
USE
WASTE


=========================START WT3 MODEL RUN==========================
.................................

Degrees of Freedom: 0

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
.................................

Degrees of Freedom: 0

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................

BIG SPRING: DWI CHECK
=========================SYSTEM LEVEL RESULTS=========================
LCOW ($/m3): 0.44694
Total Capital Investment ($MM): 12.808
Total Annual Operating Costs ($MM/yr): 0.224
Annual Fixed Operating Cost ($MM/yr): 0.171
Annual Catalysts and Chemicals Cost ($MM/yr): 0.0
Annual Electricity Costs ($MM/yr): 0.053
Annual Other Variable Costs ($MM/yr): 0.0
Treated water (m3/s): 0.075
Total water recovery (%): 99.97
Electricity intensity (kWh/m3): 

In [7]:
# Surface Discharge

case_study = 'big_spring'
scenario = 'sd_check'
desired_recovery = 1
ro_bounds = 'other' # or 'seawater'

m = watertap_setup(case_study=case_study, scenario=scenario)
m = get_case_study(m=m)
m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

sd_lcow_brine_basis = 0
for flow in flow_list:
    sd_lcow = np.array([])
    sd_flow = np.array([])
    for conc in conc_list:
        print(flow)
        m.fs.big_spring_feed.flow_vol_in.fix(flow)
        m.fs.big_spring_feed.conc_mass_in[0, 'tds'].fix(conc)
        m = run_and_return_model(m, objective=True,print_it=True)
        print(m.fs.costing.LCOW.value(),m.fs.surface_discharge.flow_vol_in[0].value)
        sd_lcow= np.concatenate((sd_lcow,[m.fs.costing.LCOW.value()]))
        sd_flow= np.concatenate((sd_flow,[m.fs.surface_discharge.flow_vol_in[0].value]))

    temp = sd_lcow*flow/sd_flow

    try: 
        sd_lcow_brine_basis = np.vstack((sd_lcow_brine_basis,temp))
    except ValueError:
        sd_lcow_brine_basis = temp


Case Study = BIG SPRING
Scenario = SD CHECK


=========================ADDING UNIT PROCESSES=========================
SOURCE
SURFACE DISCHARGE
USE
WASTE


=========================START WT3 MODEL RUN==========================
.................................

Degrees of Freedom: 0

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
.................................

Degrees of Freedom: 0

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................

BIG SPRING: SD CHECK
=========================SYSTEM LEVEL RESULTS=========================
LCOW ($/m3): 0.05583
Total Capital Investment ($MM): 1.517
Total Annual Operating Costs ($MM/yr): 0.033
Annual Fixed Operating Cost ($MM/yr): 0.02
Annual Catalysts and Chemicals Cost ($MM/yr): 0.0
Annual Electricity Costs ($MM/yr): 0.013
Annual Other Variable Costs ($MM/yr): 0.0
Treated water (m3/s): 0.075
Total water recovery (%): 99.97
Electricity intensity (kWh/m3): 0.102


In [8]:
# Brine concentrator

case_study = 'big_spring'
scenario = 'bc_check'
desired_recovery = 1
ro_bounds = 'other' # or 'seawater'

m = watertap_setup(case_study=case_study, scenario=scenario)
m = get_case_study(m=m)
m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

bc_lcow_brine_basis = 0
for flow in flow_list:
    bc_lcow = np.array([])
    bc_flow = np.array([])
    for conc in conc_list:
        print(flow)
        m.fs.big_spring_feed.flow_vol_in.fix(flow)
        m.fs.big_spring_feed.conc_mass_in[0, 'tds'].fix(conc)
        m = run_and_return_model(m, objective=True,print_it=True)
        print(m.fs.costing.LCOW.value(),m.fs.brine_concentrator.flow_vol_in[0].value)
        bc_lcow= np.concatenate((bc_lcow,[m.fs.costing.LCOW.value()]))
        bc_flow= np.concatenate((bc_flow,[m.fs.brine_concentrator.flow_vol_in[0].value]))

    temp = bc_lcow*flow/bc_flow
    try:
        bc_lcow_brine_basis = np.vstack((bc_lcow_brine_basis,temp))
    except ValueError:
        bc_lcow_brine_basis = temp



Case Study = BIG SPRING
Scenario = BC CHECK


=========================ADDING UNIT PROCESSES=========================
SOURCE
BRINE CONCENTRATOR
USE
WASTE


=========================START WT3 MODEL RUN==========================
.................................

Degrees of Freedom: 0

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
.................................

Degrees of Freedom: 0

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................

BIG SPRING: BC CHECK
=========================SYSTEM LEVEL RESULTS=========================
LCOW ($/m3): 2.12532
Total Capital Investment ($MM): 25.874
Total Annual Operating Costs ($MM/yr): 2.84
Annual Fixed Operating Cost ($MM/yr): 0.345
Annual Catalysts and Chemicals Cost ($MM/yr): 0.0
Annual Electricity Costs ($MM/yr): 2.496
Annual Other Variable Costs ($MM/yr): 0.0
Treated water (m3/s): 0.067
Total water recovery (%): 89.982
Electricity intensity (kWh/m3): 21.

In [9]:
# Crystallizer

case_study = 'big_spring'
scenario = 'c_check'
desired_recovery = 1
ro_bounds = 'other' # or 'seawater'

m = watertap_setup(case_study=case_study, scenario=scenario)
m = get_case_study(m=m)
m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

c_lcow_brine_basis = 0

for flow in flow_list:
    c_lcow = np.array([])
    c_flow = np.array([])
    for conc in conc_list:
        print(flow)
        m.fs.big_spring_feed.flow_vol_in.fix(flow)
        m.fs.big_spring_feed.conc_mass_in[0, 'tds'].fix(conc)
        m = run_and_return_model(m, objective=True,print_it=True)
        # print(m.fs.costing.LCOW.value(),m.fs.brine_concentrator.flow_vol_in[0].value)
        c_lcow= np.concatenate((c_lcow,[m.fs.costing.LCOW.value()]))
        c_flow= np.concatenate((c_flow,[m.fs.crystallizer.flow_vol_in[0].value]))
    

    temp = c_lcow*flow/c_flow
    print(flow_list,c_flow)
    try:
        c_lcow_brine_basis = np.vstack((c_lcow_brine_basis,temp))
    except ValueError:
        c_lcow_brine_basis = temp



Case Study = BIG SPRING
Scenario = C CHECK


=========================ADDING UNIT PROCESSES=========================
SOURCE


CRYSTALLIZER
USE
WASTE


=========================START WT3 MODEL RUN==========================
.................................

Degrees of Freedom: 0

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
.................................

Degrees of Freedom: 0

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................

BIG SPRING: C CHECK
=========================SYSTEM LEVEL RESULTS=========================
LCOW ($/m3): 9.3905
Total Capital Investment ($MM): 190.993
Total Annual Operating Costs ($MM/yr): 8.671
Annual Fixed Operating Cost ($MM/yr): 2.543
Annual Catalysts and Chemicals Cost ($MM/yr): 0.0
Annual Electricity Costs ($MM/yr): 6.128
Annual Other Variable Costs ($MM/yr): 0.0
Treated water (m3/s): 0.071
Total water recovery (%): 94.981
Electricity intensity (kWh/m3): 50.052
Electricity portion of LCOW (%): 29.049

=========================UNIT PROCESS RESULTS=========================


Source:
	Total

In [10]:
# Evaporation pond

case_study = 'big_spring'
scenario = 'ep_check'
desired_recovery = 1
ro_bounds = 'other' # or 'seawater'

m = watertap_setup(case_study=case_study, scenario=scenario)
m = get_case_study(m=m)
m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

ep_lcow_brine_basis = 0

for flow in flow_list:
    ep_lcow = np.array([])
    ep_flow = np.array([])
    for conc in conc_list:
        print(flow)
        m.fs.big_spring_feed.flow_vol_in.fix(flow)
        m.fs.big_spring_feed.conc_mass_in[0, 'tds'].fix(conc)
        m = run_and_return_model(m, objective=True,print_it=True)
        # print(m.fs.costing.LCOW.value(),m.fs.brine_concentrator.flow_vol_in[0].value)
        ep_lcow= np.concatenate((ep_lcow,[m.fs.costing.LCOW.value()]))
        ep_flow= np.concatenate((ep_flow,[m.fs.evaporation_pond.flow_vol_in[0].value]))

    temp = ep_lcow*flow/ep_flow
    try:
        ep_lcow_brine_basis = np.vstack((ep_lcow_brine_basis,temp))
    except ValueError:
        ep_lcow_brine_basis = temp



Case Study = BIG SPRING
Scenario = EP CHECK


=========================ADDING UNIT PROCESSES=========================
SOURCE
EVAPORATION POND
USE
WASTE


=========================START WT3 MODEL RUN==========================
.................................

Degrees of Freedom: 0

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
.................................

Degrees of Freedom: 0

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................

BIG SPRING: EP CHECK
=========================SYSTEM LEVEL RESULTS=========================
LCOW ($/m3): 0.62189
Total Capital Investment ($MM): 16.889
Total Annual Operating Costs ($MM/yr): 0.225
Annual Fixed Operating Cost ($MM/yr): 0.225
Annual Catalysts and Chemicals Cost ($MM/yr): 0.0
Annual Electricity Costs ($MM/yr): 0.0
Annual Other Variable Costs ($MM/yr): 0.0
Treated water (m3/s): 0.067
Total water recovery (%): 89.982
Electricity intensity (kWh/m3): 0.0
El

In [11]:
# Landfill

case_study = 'big_spring'
scenario = 'lf_check'
desired_recovery = 1
ro_bounds = 'other' # or 'seawater'

m = watertap_setup(case_study=case_study, scenario=scenario)
m = get_case_study(m=m)
m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

lf_lcow_brine_basis = 0

for flow in flow_list:
    lf_lcow = np.array([])
    lf_flow = np.array([])
    for conc in conc_list:
        print(flow)
        m.fs.big_spring_feed.flow_vol_in.fix(flow)
        m.fs.big_spring_feed.conc_mass_in[0, 'tds'].fix(conc)
        m = run_and_return_model(m, objective=True,print_it=True)
        lf_lcow= np.concatenate((lf_lcow,[m.fs.costing.LCOW.value()]))
        lf_flow= np.concatenate((lf_flow,[m.fs.landfill_zld.flow_vol_in[0].value]))

    temp = lf_lcow*flow/lf_flow
    try:
        lf_lcow_brine_basis = np.vstack((lf_lcow_brine_basis,temp))
    except ValueError:
        lf_lcow_brine_basis = temp



Case Study = BIG SPRING
Scenario = LF CHECK


=========================ADDING UNIT PROCESSES=========================
SOURCE
LANDFILL ZLD
USE
WASTE


=========================START WT3 MODEL RUN==========================
.................................

Degrees of Freedom: 0

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
.................................

Degrees of Freedom: 0

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................

BIG SPRING: LF CHECK
=========================SYSTEM LEVEL RESULTS=========================
LCOW ($/m3): 0.03832
Total Capital Investment ($MM): 1.156
Total Annual Operating Costs ($MM/yr): 0.015
Annual Fixed Operating Cost ($MM/yr): 0.015
Annual Catalysts and Chemicals Cost ($MM/yr): 0.0
Annual Electricity Costs ($MM/yr): 0.0
Annual Other Variable Costs ($MM/yr): 0.0
Treated water (m3/s): 0.075
Total water recovery (%): 99.97
Electricity intensity (kWh/m3): 0.0
Electric

In [12]:
# plt.plot(dwi_flow,dwi_lcow_brine_basis,label='DWI')
# plt.plot(sd_flow,sd_lcow_brine_basis,label='Surface Discharge')
# plt.plot(bc_flow,bc_lcow_brine_basis,label='Brine Concentrator')
# plt.plot(c_flow,c_lcow_brine_basis,label='Crystallizer')
# plt.plot(ep_flow,ep_lcow_brine_basis,label='Evaporation Pond')
# plt.plot(lf_flow,lf_lcow_brine_basis,label='Landfill ZLD')

# plt.legend()
# plt.xlabel('Flow (m3 brine/s)')
# plt.ylabel('LCOW ($/m3 brine)')
# plt.yscale('log')

In [13]:
dwi_lcow_brine_basis

array([[0.86962245, 0.86962245, 0.86962245],
       [0.32624843, 0.32624843, 0.32624843],
       [0.27041837, 0.27041837, 0.27041837]])

In [14]:
sd_lcow_brine_basis

array([[0.07289926, 0.07289926, 0.07289926],
       [0.04919693, 0.04919693, 0.04919693],
       [0.04560594, 0.04560594, 0.04560594]])

In [15]:
bc_lcow_brine_basis

array([[ 2.05291814, 15.54467143, 29.03642473],
       [ 2.19288572,  3.2317705 ,  4.27065528],
       [ 2.19703645,  3.02837342,  3.85971039]])

In [16]:
c_lcow_brine_basis

array([[10.5001844 , 10.56867318, 10.63716195],
       [ 9.28511533,  9.38137891,  9.47764248],
       [ 9.22605709,  9.32278358,  9.41951007]])

In [17]:
ep_lcow_brine_basis

array([[0.71387407, 0.71387407, 0.71387407],
       [0.60378585, 0.60378585, 0.60378585],
       [0.59679927, 0.59679927, 0.59679927]])

In [18]:
lf_lcow_brine_basis

array([[0.0765556 , 0.07965486, 0.08270327],
       [0.02745914, 0.0285708 , 0.02966421],
       [0.02241463, 0.02332206, 0.0242146 ]])